# Combine Raw Big Query Files

This notebook takes in zip file folders of Reddit comments and/or Reddit submissions as directly downloaded from the Google Big Query data source in order to output smaller output files that only contain the subreddits of interest to be used in analysis.

First, get your data.

Files and folders will begin with:
- RC_YYYY_MM for reddit comments  
- RS_YYYY_MM for reddit submissions (posts)

Your files should be stored as zip files (**DO NOT** manually unzip files) with the following hierarchy:

**raw_data_folder**  
$\hspace{10mm}$**Rx_YYYY_MM**   <- (folder)  
$\hspace{20mm}$**Rx_YYYY_MM_docXX.gz**   <- (gzipped csv file)  
$\hspace{20mm}$**Rx_YYYY_MM_docXX.gz**   <- (gzipped csv file)  
$\hspace{20mm}$...  
$\hspace{10mm}$**Rx_YYYY_MM** <- (folder)  
$\hspace{20mm}$**Rx_YYYY_MM_docXX.gz**   <- (gzipped csv file)  
$\hspace{20mm}$**Rx_YYYY_MM_docXX.gz**   <- (gzipped csv file)  
$\hspace{20mm}$...  
$\hspace{10mm}$...   

In [1]:
#imports
import pandas as pd
import csv
import gzip
import os
import numpy as np
from collections import defaultdict
import time
import shutil
from math import floor

In [2]:
#helper functions
def get_immediate_subdirectories(a_dir):
    subdirectories = [name for name in os.listdir(a_dir) 
                      if os.path.isdir(os.path.join(a_dir, name)) 
                      and not name.startswith('.')]
    return sorted(subdirectories)

def get_files_in_dir(a_dir):
    return sorted([os.path.join(a_dir,f) for f in os.listdir(a_dir) if not f.startswith('.')])

def get_filepaths_in_folders(raw_data_folder):
    infolders = get_immediate_subdirectories(raw_data_folder)

    filepaths = {folder:get_files_in_dir(os.path.join(raw_data_folder,folder)) for folder in infolders}
    return filepaths

In [3]:
def parse_files(raw_data_folder, outfolder, preview):
    
    ### WARNING: this will overwrite any existing folders/files of the same name as the output folder
    start = time.time()
    
    print(round(time.time()-start),'seconds','\tGetting file names')
    files = get_filepaths_in_folders(raw_data_folder)
    
    if not os.path.exists(outfolder):
        os.makedirs(outfolder)
        
    subreddits = ['canada', 'politics', 'Republican', 'The_Donald', 'Catholicism', 
              'news', 'worldnews', 'Libertarian', 'PoliticalDiscussion', 'communism',
              'unitedkingdom', 'TrueReddit', 'Conservative', 'college', 'socialism',
              'TheNewRight', 'geopolitics', 'technology', 'environment', 'neutralnews',
              'Anarchism', 'NeutralPolitics', 'worldpolitics', 'democrats', 'Liberal',
              'progressive', 'Full_news', 'moderatepolitics', 'qualitynews', 'worldevents', 
              'internationalpolitics']
        
    
    for folder in files:
        print('\n----- Folder: {} -----'.format(folder))
        if not os.path.exists(os.path.join(outfolder,folder)):
            os.makedirs(os.path.join(outfolder,folder))
        print(round(time.time()-start),'seconds','\tReading files in folder')
        
        if folder[0:2] == 'RC':
            columns = ['body', 'subreddit', 'link_id']
        elif folder[0:2] == 'RS':
            columns = ['subreddit', 'id','title','domain', 'url','selftext', 'is_self']
            #title,subreddit,created_utc,num_comments,score,selftext,id,domain,url,is_self,permalink

        df = pd.concat((pd.read_csv(file, compression='gzip', header=0, sep=',', quotechar='"',
                        usecols = columns) 
                        for file in files[folder]))

        
        for sub in subreddits:
            outfile_name = str(folder)+'_'+ str(sub) +'.csv'
            print(round(time.time()-start),'seconds','\tWriting',outfile_name)
            
            subdf = df[df['subreddit'] == sub]
            subdf = subdf.replace('�','',regex=True)
            subdf = subdf.replace('"',"'",regex=True)
            subdf = subdf.replace('&gt;','>',regex=True)
            subdf = subdf.replace('&lt;','>', regex=True)
            subdf = subdf.replace('&amp;','&', regex=True) 
            subdf = subdf.replace('&nbsp;','&', regex=True)
            subdf = subdf.replace('&#x200b;','&', regex=True)
            subdf = subdf.replace('\s+',' ',regex=True)
            subdf = subdf.replace('\n*','',regex=True)
            subdf = subdf.replace('\[.+\]\([^\s]+\)','',regex=True)
            subdf = subdf.replace('http\w*:[^\s]*','',regex=True)
            subdf = subdf.replace('& ','',regex=True)
            subdf = subdf.replace('> ','', regex=True)
            
            
            
            
            if preview == True:
                display(subdf.head(15))
            f = os.path.join(outfolder,folder,outfile_name)
            subdf.to_csv(f, header=True, index=False,quotechar='"')   
            
            if os.path.getsize(f)/1000000 > 100:
                print(round(time.time()-start),'seconds','\tFile {} is too large (>100MB).  Splitting into multiple documents.'.format(outfile_name))
                maxlines = 400000
                with open(f) as f_in:
                    i = 0
                    docnum = 0
                    outfile_name = str(folder)+'_'+ str(sub) + '_doc' + str(docnum) +'.csv'
                    f_out = open(os.path.join(outfolder,folder,outfile_name),'w')
                    for line in f_in:
                        f_out.write(line)
                        i += 1
                        if i%maxlines ==0 and i != 0:
                            f_out.close()
                            docnum += 1
                            outfile_name = str(folder)+'_'+ str(sub) + '_doc' + str(docnum) +'.csv'
                            f_out = open(os.path.join(outfolder,folder,outfile_name),'w')
                    f_out.close()
                os.remove(f) 
        
        end = time.time()
        print('\n{} minutes {} seconds'.format(floor((end-start)/60), round((end-start)%60)))
        
        
    print('\nscript complete')
    


In [4]:
##fill out all variables in this cell

# #set path to the raw BigQuery data folder
raw_data_folder = '../00-GettingData/ExampleRawBigQueryFiles'

#within the github folder, set the output folder where you would like output files to be saved
# ##### WARNING: if the folders/files already exist, the function will overwrite existing files! #####
outfolder = 'ParsedBigQueryData'

#do you want the script to show previews of the output data?
preview = False

In [5]:
##run this cell as-is
parse_files(raw_data_folder, outfolder, preview)

0 seconds 	Getting file names

----- Folder: RC_2018_12 -----
0 seconds 	Reading files in folder
23 seconds 	Writing RC_2018_12_canada.csv
39 seconds 	Writing RC_2018_12_politics.csv
181 seconds 	File RC_2018_12_politics.csv is too large (>100MB).  Splitting into multiple documents.
183 seconds 	Writing RC_2018_12_Republican.csv
184 seconds 	Writing RC_2018_12_The_Donald.csv
231 seconds 	File RC_2018_12_The_Donald.csv is too large (>100MB).  Splitting into multiple documents.
232 seconds 	Writing RC_2018_12_Catholicism.csv
238 seconds 	Writing RC_2018_12_news.csv
288 seconds 	File RC_2018_12_news.csv is too large (>100MB).  Splitting into multiple documents.
289 seconds 	Writing RC_2018_12_worldnews.csv
363 seconds 	File RC_2018_12_worldnews.csv is too large (>100MB).  Splitting into multiple documents.
364 seconds 	Writing RC_2018_12_Libertarian.csv
378 seconds 	Writing RC_2018_12_PoliticalDiscussion.csv
381 seconds 	Writing RC_2018_12_communism.csv
382 seconds 	Writing RC_2018_12_uni